

# Toward self-documenting, customizable microbiome analysis and metaanalysis with scikit-bio, QIIME 2 and Qiita.

Greg Caporaso and Rob Knight

**[Caporaso Lab](http://caporasolab.us), Northern Arizona University**

**[Knight Lab](http://https://knightlab.ucsd.edu/), University of California San Diego**

# QIIME

* Cited 2969 times (as of 16 July 2015) since publication.

* Nearly monthly workshops, somewhere in the world.

* Biggest issues:
 * Hard to install, command line interface is challenging for many users.
 * Workflows can be "black boxes".
 * ``my-favoriite-tool`` isn't available in QIIME

## scikit-bio

low-level, a bioinformatics *library* for data scientists, students, and bioinformatics software developers

free, Sloan-funded companion text, *An Introduction to Applied Bioinformatics* available at [readIAB.org](http://www.readIAB.org)




## QIIME 2

a stable and well-documented API and command line interface
 * enables self-documenting analysis, integration as a component of other systems, and plugin development
 * supports analysis in cloud and cluster environments




## Qiita

end-user, browser-based environment for microbiome analysis and meta-analysis

## I/O in bioinformatics is hard

- format redundancy (many-to-many)

- format ambiguity

- heterogeneous sources

## Format redundancy (many-to-many)

In [ ]:
from skbio import DNA

seq1 = DNA.read('data/seqs.fasta', qual='data/seqs.qual')
seq2 = DNA.read('data/seqs.fastq', variant='illumina1.8')
seq1

In [ ]:
seq1 == seq2

## Format ambiguity

In [ ]:
import skbio.io

skbio.io.sniff('data/mystery_file.bz2')

## Heterogeneous sources

#### Read a gzip file from a URL:

In [ ]:
from skbio import TreeNode

tree1 = skbio.io.read('http://localhost:8888/files/data/newick.gz', 
                      into=TreeNode)
print(tree1.ascii_art())

## We are in beta - should you even use our software?

#YES!

## API Lifecycle
![](assets/stability-state-diagram.svg)


### What is stable:

- `skbio.io` 
- `skbio.sequence`

&nbsp;
&nbsp;
###What is next:

- `skbio.alignment`
- `skbio.tree`
- `skbio.diversity`
- `skbio.stats`
- &lt;`your awesome subpackage!`&gt;

## Sequence API: putting the *scikit* in scikit-bio

In [ ]:
seq = DNA("ACGTCCAGGTACCAGGTACCacgtacgtacgtacgtGTTTTACTGACGCAGGACGTACggggacaccgac", lowercase='exon')
seq

## Made with numpy

In [ ]:
seq.values

## And a pinch of pandas

In [ ]:
seq.positional_metadata

## Slicing with positional metadata:

In [ ]:
seq[seq.positional_metadata['exon']]

# Many other mature scientific computing tools

* Beautiful visualizations with seaborn

* Powerful machine learning and optimization with scikit-learn

* Fast sequence collapsing with marisa-trie and related packages

* Statistical modeling with Stats.Models

## Example: building a better taxonomy classifier in under 100 lines of code

In [ ]:
from skbio import DNA
import numpy as np
import skbio

## Load Greengenes and slice all sequences to the region 
## amplified by 515F/806R. 

aligned_seqs_fp = 'data/gg_13_8_otus/rep_set_aligned/82_otus.fasta'
taxonomy_fp = 'data/gg_13_8_otus/taxonomy/82_otu_taxonomy.txt'


fwd_primer = DNA("GTGCCAGCMGCCGCGGTAA",
                 metadata={'label':'fwd-primer'})
rev_primer = DNA("GGACTACHVGGGTWTCTAAT",
                 metadata={'label':'rev-primer'}).reverse_complement()

def seq_to_regex(seq):
    result = []
    for base in str(seq):
        if base in DNA.degenerate_chars:
            result.append('[{0}]'.format(
                ''.join(DNA.degenerate_map[base])))
        else:
            result.append(base)

    return ''.join(result)

regex = '({0}.*{1})'.format(seq_to_regex(fwd_primer),
                            seq_to_regex(rev_primer))

starts = []
stops = []
for seq in skbio.io.read(aligned_seqs_fp, format='fasta', 
                         constructor=DNA):
    for match in seq.find_with_regex(regex, ignore=seq.gaps()):
        starts.append(match.start)
        stops.append(match.stop)
        
locus = slice(int(np.median(starts)), int(np.median(stops)))

## Get all kmers for all sequences.
kmer_counts = []
seq_ids = []
for seq in skbio.io.read(aligned_seqs_fp, format='fasta',
                         constructor=DNA):
    seq_ids.append(seq.metadata['id'])
    sliced_seq = seq[locus].degap()
    kmer_counts.append(sliced_seq.kmer_frequencies(8))
    
    
## Load the training/test data, perform feature selection, and train 
## and test the classifier using cross-validation.
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_selection import SelectPercentile
from sklearn.svm import SVC
from sklearn.cross_validation import train_test_split

X = DictVectorizer().fit_transform(kmer_counts)

taxonomy_level = 3 # class
id_to_taxon = {}
with open(taxonomy_fp) as f:
    for line in f:
       id_, taxon = line.strip().split('\t')
       id_to_taxon[id_] = '; '.join(taxon.split('; ')[:taxonomy_level])

y = [id_to_taxon[seq_id] for seq_id in seq_ids]

X = SelectPercentile().fit_transform(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=0)
y_pred = SVC(C=10, kernel='linear', degree=3,
             gamma=0.001).fit(X_train, y_train).predict(X_test)

## Define a function to use for plotting. 
%matplotlib inline
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.figure()
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    plt.ylabel('Known taxonomy')
    plt.xlabel('Predicted taxonomy')
    plt.tight_layout()
    plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score

cm = confusion_matrix(y_test, y_pred)
cm_normalized = cm / cm.sum(axis=1)[:, np.newaxis]
plot_confusion_matrix(cm_normalized, title='Normalized confusion matrix')

print("F-score: %1.3f" % f1_score(y_test, y_pred, average='micro'))

## Support Vector Machine genus classifier

![](./svc-genus.png)

## Naive Bayes genus classifier

![](./naive-bayes-genus.png)

# QIIME 2

Self-documenting analyses simplify bioinformatics methods reporting, and therefore reproducibility and transparancy.

Stable, documented API enables integration with other platforms (e.g., Illumina Basespace) and plugin development.

Many ongoing method benchmarks will inform core methods, e.g.,
 * "OTU-free" analyses
 * improved taxonomic classification
 * diverse machine learning approaches for sequence and sample classification

Easier parallel deployment with the IPython parallel framework